# Question 2
Last homework we found a method to find the least-squares solution using gradient descent with a constant step size. In this code, we will compare the closed form solution to the iteratively solved solution.

In [1]:
#Run these to import all of the necessary files
import numpy as np
import scipy as sp
import scipy.io as spio
import scipy.signal as sig
import scipy.io.wavfile as wf
import matplotlib as plt
from pylab import *
import os
import re
import time

# Plots graphs in the browser window
%matplotlib inline

In [2]:
#generate A matrix and y vector to use least squares regression
#this code takes a while to run
np.random.seed(121234)
A = np.random.rand(10000,5000)
u,s,vh = np.linalg.svd(A)
diagS = np.diagflat((sqrt(40)+(sqrt(50)-sqrt(40))*np.random.rand(A.shape[1],1)))
diagS2 =zeros((A.shape[0]-A.shape[1],A.shape[1]))
S = np.append(diagS,diagS2,axis=0)
A = (u.dot(S)).dot(vh)
y = np.random.rand(A.shape[0],1)

## a)
create the function LS_closed_sol which takes in a matrix A and vector y and returns the square error using the closed form solution of least squares regression. 
As a reminder, the squared error is: $$||A\vec{x}-\vec{y}||^2$$
The closed form solution for $\vec{x}$ is: $$\vec{x} = (A^TA)^{-1}A^T\vec{y}$$

In [3]:
def LS_closed_sol(A,y):
    AT = A.T
    ATA = A.T.dot(A)
    x = inv(ATA).dot(AT).dot(y)
    e2 = pow(norm(A.dot(x)-y),2)
    return e2

## b)
The first step to creating the iteratively solved least squares model is to find a stable step size $\alpha$.

Create the function alpha_power_iter(A,v,iters) which takes as input a matrix A, a vector v, and a constant iters. The function should find the maximum eigenvalue of $A^TA$ using power iterations, with v being the initial condition vector and iters being the number of iterations (disc8B goes over power iteration). The function should return the value $\alpha$ where $\alpha = \frac{1}{\lambda_{max}}$. Note that we can't use the optimal $\alpha$ since power iteration only allows us to find the maximum eigenvalue and not the minimum.

In [4]:
def alpha_power_iter(A, v, iters):
    at = A.T
    ata = at.dot(A)
    for i in range(iters):
        v = ata.dot(v)/norm(ata.dot(v))
    eig_mat = ata.dot(v)
    eig_max = eig_mat[0]/v[0]
    alpha = 1/eig_max
    return alpha

## C)
Now that we have our step size $\alpha$, we can create our function to iteratively solve for $\vec{x}$.

Create a function LS_iter(A,y,alpha,x2,iters) which takes as input a matrix A, vector y, constant alpha, vector x2, and constant iters. The function should calculate $\vec{x}(t+1) = \vec{x}(t)-\alpha A^T(A\vec{x}-\vec{y})$ up to time step $\vec{x}(\text{iters})$ as well as the squared error at each time step. The function should return an array which contains the squared error $||A\vec{x}(t)-\vec{y}||^2$ for each time step.

In [5]:
def LS_iter(A,y,alpha,x2,iters):
    at = A.T
    ata = at.dot(A)
    alata = alpha*ata
    alaty = alpha*at.dot(y)
    e2 = []
    for i in range(iters):
        x2 = x2-alata.dot(x2)+alaty
        e2.append(pow(norm(A.dot(x2)-y),2))
    return e2

# D)
Now let's compare the closed form solution to the iteratively solved solution. Using the A and y given, as well as the initial vectors v and x2, use the functions we just defined to solve for the squared error using both the iterative method and closed form solution. For the iterative method, use 5 iterations for finding $\alpha$ and 10 iterations to calculate the squared error.

Compare the time it takes to solve using the iterative method versus the closed form solution. Hint: the function time.time() will give the current time in seconds.

For the iterative method, plot the squared error versus the number of time steps elapsed for $\vec{x}(t)$. On the same graph, plot a constant horizontal line at the value of the closed solution's squared error to compare the differences.

In [6]:
np.random.seed(13123)
v = np.random.rand(A.shape[1],1)
np.random.seed(34121)
x2 = np.zeros((A.shape[1],1))

start_cf=time.time()
e2 = LS_closed_sol(A,y)
end_cf = time.time()
duration_cf = end_cf-start_cf

start_iter = time.time()
alpha = alpha_power_iter(A,v,5)
iters = 10
ee2= LS_iter(A,y,alpha,x2,iters)
end_iter = time.time()
duration_iter = end_iter-start_iter

print('Time elapsed for closed form solution:',duration_cf)
print('Time elapsed for iterative solution:',duration_iter)


plt_iters = linspace(1,iters,num=iters)
plt_e2 = [e2]*len(plt_iters)
line1,=plt.plot(plt_iters,ee2)
line2,=plt.plot(plt_iters,plt_e2)
plt.legend([line1, line2], ['iterative solution', 'closed form solution'])

Time elapsed for closed form solution: 9.03472900390625
Time elapsed for iterative solution: 5.08156681060791
